# 2P-Fairdivision-IndivisibleItems

## Introduction
The aim of this project is to benchmark some alogorithm design to allocate items to two agents. This notebook focuses on same-size allocations, meaning that each agent should receive the same number of goods. As an additional constraint, items are not divisible.

This notebook is inspired by D. Marc Kilgour and Rudolf Vetschera article named *Comparing direct algoriths for two-player fair division of indivisible items - A computational study* you can found [here](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2997431)

## How to use this notebook
This notebook shows how to use algorithms and get properties of returned allocations. Several classes are defined to help you to create your own algorithms and properties :
  * Agent, which store preferences of an agent as long as some helper methods like getting the to ranked item, get the Borda score of a provided allocation ... Take a look to see what can be done !
  * Good, which is a simple wrapper to represent items. It has some nice printing  and comparison methods
  * ...
In the `properties` module, you can find a bunch of functions. They are designed to test several allocation properties such as Pareto-Optimality, Max-Min property and more.

## First steps
Let's create two agents with their preferences. As a start, we will also create 4 items to divide among agents.

In [2]:
# Manipulate path
import os
import sys
sys.path.insert(0, os.path.join(os.getcwd(), 'fairdiv'))

# Import fairdiv
import fairdiv


# Create some constants
NUMBER_OF_GOODS = 4

# Create items
items = [fairdiv.Good(i) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[0], items[2], items[1], items[3]])
]

Once items and agents are created, you can use one of the available algorithm to compute an allocation. Let's use a sequential algorithm named `bottom_up`.
This algorithm generates two allocations, one when agent A is the first agent to pick an item and an other when it is agent B.

In [3]:
# Import algorithms
import fairdiv.algorithm as algorithm


# Use bottom-up to compute two allocations
(x1, x2) = algorithm.bottom_up(agents, items)
print(x1)
print(x2)

{B: (2, 3), A: (0, 1)}
{B: (0, 2), A: (1, 3)}


Once you generated some allocations, you can test different properties with the `properties` module. Let's benchmark our allocations `x1` and `x2` to see how beautiful they are !

In [4]:
# Import module
import fairdiv.properties as properties


# Test some of them
print(properties.is_borda_envy_free(x1, agents))
print(properties.is_borda_envy_free(x2, agents))

False
False


Some properties need to get all possible allocations for a given item set size. To generate them, use `Allocation.generate_all_allocations`.

In [5]:
# Generate all allocations for our problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))
print(A)

# Use them in properties
print(properties.is_pareto(x1, A, agents))
print(properties.is_pareto(x2, A, agents))

[{B: (0, 2), A: (1, 3)}, {B: (1, 2), A: (0, 3)}, {B: (0, 1), A: (2, 3)}, {B: (1, 3), A: (0, 2)}, {B: (2, 3), A: (0, 1)}, {B: (0, 3), A: (1, 2)}]
True
True


## Test a large amount of properties
Because the number of properties to test is possibly hudge, we designed a class called `Statistics` with which you can store computed allocations and automatically test a bunch of properties easly.
First you need to create a new `Statistics` object.

In [6]:
import statistics as stats


# Create a new Statistics object for storing allocations from bottom_up
bu_stats = stats.Statistics(A, agents, {
    "is_borda_envy_free": lambda X, A, M: properties.is_borda_envy_free(X, M),
    "is_envy_free": lambda X, A, M: properties.is_envy_free(X, M),
    "is_borda_max_min": properties.is_borda_max_min,
    "is_borda_nash": properties.is_borda_nash
})

Now you can add allocations to your object. Each time a new allocation is added, desired properties are tested and the result is stored. You can then print the result or save it in a file.

In [7]:
# Add allocations
bu_stats.add(x1)
bu_stats.add(x2)

# Print the result
print(bu_stats.formatted_text())

Allocations:
	Allocation: {B: (2, 3), A: (0, 1)}
		is_envy_free: True
		is_borda_envy_free: False
		is_borda_max_min: False
		is_borda_nash: True
	Allocation: {B: (0, 2), A: (1, 3)}
		is_envy_free: True
		is_borda_envy_free: False
		is_borda_max_min: False
		is_borda_nash: True



## Reproducing examples
In this section, we are going to test our algorithm using problems defined in the article. This will allow us to demonstrate that our algorithm are correct.

### First Problem
Benchmarking OS algorithm with the given problem :
  * 4 items
  * B preferences are $1 \succ 3 \succ 2 \succ 4$

The algorithm should produce 6 allocations, but only 4 unique ones.

In [4]:
import fairdiv
import statistics as stats
import fairdiv.algorithm as algorithm
import fairdiv.properties as props


# Create some constants
NUMBER_OF_GOODS = 4

# Create items
items = [fairdiv.Good(i+1) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[0], items[2], items[1], items[3]])
]

# Generate all allocations for the given problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))
for alloc in A:
    print('{} -> {}'.format(alloc, props.is_envy_free_ordinally(alloc, agents)))

result = algorithm.original_sequential(agents, items)
print("OS: {}".format(result))

TR = algorithm.trump_algorithm(agents, items)
print("TR: {}".format(TR))
for alloc in TR:
    print(props.is_pareto_ordinally(alloc, A, M))


{B: (2, 3), A: (1, 4)} -> True
{B: (1, 4), A: (2, 3)} -> True
{B: (1, 2), A: (3, 4)} -> False
{B: (3, 4), A: (1, 2)} -> False
{B: (2, 4), A: (1, 3)} -> False
{B: (1, 3), A: (2, 4)} -> False
OS: [{B: (1, 3), A: (2, 4)}, {B: (1, 4), A: (2, 3)}, {B: (2, 3), A: (1, 4)}, {B: (3, 4), A: (1, 2)}]
TR: []


### Second problem
We are benchmarking all algorithm with the given problem:
  * 8 items
  * B's ranking is : $2 \succ 4 \succ 5 \succ 6 \succ 7 \succ 8 \succ 1 \succ 3$

Results should be :
  * OS is the only algorithm that generates allocation `(1,3,5,6)|(2,4,7,8)`
  * Only OS algoithm can allocate $5$ and $6$ to the same player
  * Allocation `(1,3,4,7)|(2,5,6,8)` is not MM (max-min)


In [2]:
import fairdiv

import os
import sys
sys.path.insert(0, os.path.join('.', 'fairdiv'))

import statistics as stats
import fairdiv.algorithm as algorithm
import fairdiv.properties as properties


# Create some constants
NUMBER_OF_GOODS = 8

# Create items
items = [fairdiv.Good(i+1) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[1], items[3], items[4], items[5], items[6], items[7], items[0], items[2]])
]

# Generate all allocations for the given problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))


results = {
    "OS": algorithm.original_sequential(agents, items),
    "RS": algorithm.restricted_sequential(agents, items),
    "SD": algorithm.singles_doubles(agents, items),
    "BU": algorithm.bottom_up(agents, items),
    "TR": algorithm.trump_algorithm(agents, items)
}

for k, v in results.items():
    print("{}: {}".format(k, v))

print("\nMax-min property for OS allocations:")
for alloc in results["OS"]:
    print("\t{}: {}".format(alloc, properties.is_max_min(alloc, A, agents)))

TA: {{A: (1, 3, 4, 6), B: (2, 5, 7, 8)}}
SD: {{A: (1, 3, 4, 6), B: (2, 5, 7, 8)}, {A: (1, 3, 4, 5), B: (2, 6, 7, 8)}, {A: (1, 2, 3, 6), B: (4, 5, 7, 8)}, {A: (1, 2, 3, 5), B: (4, 6, 7, 8)}}
RS: {{A: (1, 3, 5, 7), B: (2, 4, 6, 8)}}
OS: {{A: (1, 3, 5, 7), B: (2, 4, 6, 8)}, {A: (1, 3, 4, 7), B: (2, 5, 6, 8)}, {A: (1, 3, 5, 6), B: (2, 4, 7, 8)}}
BU: {{A: (1, 3, 4, 6), B: (2, 5, 7, 8)}, {A: (1, 2, 3, 5), B: (4, 6, 7, 8)}}

Max-min property for OS allocations:
	{A: (1, 3, 5, 7), B: (2, 4, 6, 8)}: False
	{A: (1, 3, 4, 7), B: (2, 5, 6, 8)}: False
	{A: (1, 3, 5, 6), B: (2, 4, 7, 8)}: True


### Third Problem
The third problem is the following :
  * 8 items
  * B's ranking is $4 \succ 7 \succ 2 \succ 3 \succ 6 \succ 1 \succ 8 \succ 5$

Results should be :
  * BU is the only algorithm that find allocation `(1,3,5,6)|(2,4,7,8)`
  * All to-down algorithm allocate item $2$ to $A$


In [1]:
import fairdiv

import os
import sys
sys.path.insert(0, os.path.join('.', 'fairdiv'))

import statistics as stats
import fairdiv.algorithm as algorithm
import fairdiv.properties as properties


# Create some constants
NUMBER_OF_GOODS = 8

# Create items
items = [fairdiv.Good(i+1) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[3], items[6], items[1], items[2], items[5], items[0], items[7], items[4]])
]

# Generate all allocations for the given problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))


results = {
    "OS": algorithm.original_sequential(agents, items),
    "RS": algorithm.restricted_sequential(agents, items),
    "SD": algorithm.singles_doubles(agents, items),
    "BU": algorithm.bottom_up(agents, items),
    "TR": algorithm.trump_algorithm(agents, items)
}

for k, v in results.items():
    print("{}: {}".format(k, v))

RS: {{A: (1, 2, 3, 5), B: (4, 6, 7, 8)}, {A: (1, 2, 5, 6), B: (3, 4, 7, 8)}}
BU: {{A: (1, 3, 5, 6), B: (2, 4, 7, 8)}, {A: (1, 2, 3, 5), B: (4, 6, 7, 8)}}
OS: {{A: (1, 2, 3, 5), B: (4, 6, 7, 8)}, {A: (1, 2, 5, 6), B: (3, 4, 7, 8)}}
TA: {{A: (1, 2, 3, 5), B: (4, 6, 7, 8)}}


### Fourth problem
The problem is the following :
  * 8 items
  * B's ranking is : $2 \succ 5 \succ 6 \succ 1 \succ 7 \succ 3 \succ 8 \succ 4$

We expect the following results :
  * SD is the only algorithm that find allocation `(1,3,4,6)|(2,5,7,8)`
  * RS and OS allocate item 6 to player $B$
  * other algorithm allocate items 6, 7 and 8 to $B$


In [3]:
import fairdiv

import os
import sys
sys.path.insert(0, os.path.join('.', 'fairdiv'))

import statistics as stats
import fairdiv.algorithm as algorithm
import fairdiv.properties as properties


# Create some constants
NUMBER_OF_GOODS = 8

# Create items
items = [fairdiv.Good(i+1) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[1], items[4], items[5], items[0], items[6], items[2], items[7], items[3]])
]

# Generate all allocations for the given problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))


results = {
    "OS": algorithm.original_sequential(agents, items),
    "RS": algorithm.restricted_sequential(agents, items),
    "SD": algorithm.singles_doubles(agents, items),
    "BU": algorithm.bottom_up(agents, items),
    "TR": algorithm.trump_algorithm(agents, items)
}

for k, v in results.items():
    print("{}: {}".format(k, v))

RS: {{A: (1, 3, 4, 7), B: (2, 5, 6, 8)}}
BU: {{A: (1, 3, 4, 5), B: (2, 6, 7, 8)}, {A: (1, 2, 3, 4), B: (5, 6, 7, 8)}}
OS: {{A: (1, 3, 4, 7), B: (2, 5, 6, 8)}}
TA: {{A: (1, 3, 4, 5), B: (2, 6, 7, 8)}}


### Fifth Problem
Problem is :
  * 8 items
  * B's ranking is: $3 \succ 4 \succ 6 \succ 1 \succ 8 \succ 5 \succ 2 \succ 7$

We expect all algorithm should allocate 4 to $B$ and 5 to $A$

In [4]:
import fairdiv

import os
import sys
sys.path.insert(0, os.path.join('.', 'fairdiv'))

import statistics as stats
import fairdiv.algorithm as algorithm
import fairdiv.properties as properties


# Create some constants
NUMBER_OF_GOODS = 8

# Create items
items = [fairdiv.Good(i+1) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[2], items[3], items[5], items[0], items[7], items[4], items[1], items[6]])
]

# Generate all allocations for the given problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))


results = {
    "OS": algorithm.original_sequential(agents, items),
    "RS": algorithm.restricted_sequential(agents, items),
    "SD": algorithm.singles_doubles(agents, items),
    "BU": algorithm.bottom_up(agents, items),
    "TR": algorithm.trump_algorithm(agents, items)
}

for k, v in results.items():
    print("{}: {}".format(k, v))

RS: {{A: (1, 2, 5, 7), B: (3, 4, 6, 8)}}
BU: {{A: (1, 2, 3, 7), B: (4, 5, 6, 8)}, {A: (1, 2, 5, 7), B: (3, 4, 6, 8)}}
OS: {{A: (1, 2, 5, 7), B: (3, 4, 6, 8)}}
TA: {{A: (1, 2, 5, 7), B: (3, 4, 6, 8)}}


### First Borda Problem
Problem is the following:
  * $N = 10$
  * B's ranking is: $3 \succ 4 \succ 5 \succ 6 \succ 7 \succ 8 \succ 1 \succ 2$

What we can expect is :
  * OS is the only algorithm to assign `(1,2,5,6)` to $A$
  * All algorithms except RS find allocation `(1,2,4,6)` which is not borda-max-min

In [5]:
import os
import sys
sys.path.insert(0, os.path.join('.', 'fairdiv'))

import fairdiv
import fairdiv.algorithm as algorithm
import fairdiv.properties as props


# Create some constants
NUMBER_OF_GOODS = 8

# Create items
items = [fairdiv.Good(i+1) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[2], items[3], items[4], items[5], items[6], items[7], items[0], items[1]])
]

# Generate all allocations for the given problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))


results = {
    "OS": algorithm.original_sequential(agents, items),
    "RS": algorithm.restricted_sequential(agents, items),
    "SD": algorithm.singles_doubles(agents, items),
    "BU": algorithm.bottom_up(agents, items),
    "TR": algorithm.trump_algorithm(agents, items)
}

for k, v in results.items():
    print("{}: {}".format(k, v))

print

RS: {{A: (1, 2, 5, 7), B: (3, 4, 6, 8)}}
BU: {{A: (1, 2, 4, 6), B: (3, 5, 7, 8)}, {A: (1, 2, 3, 5), B: (4, 6, 7, 8)}}
OS: {{A: (1, 2, 5, 7), B: (3, 4, 6, 8)}, {A: (1, 2, 5, 6), B: (3, 4, 7, 8)}}
TA: {{A: (1, 2, 4, 6), B: (3, 5, 7, 8)}}


The OS algorithm is not generating allocation `(1,2,4,6)|(3,5,7,8)` as expected. We don't know if it is our algorithm or if we didn't understood the article's sentence.

### Second Borda Problem
The problem is the following :
  * $N = 8$
  * B's ranking is: $5 \succ 8 \succ 2 \succ 3 \succ 4 \succ 6 \succ  1 \succ 7$

We expect the following properties:
  * TR is the only algorithm that allocates `(1,2,3,7)` to $A$
  * All other algorithms allocate `(1,2,3,7)`to $A$ and this allocation fulfill all ordinal and Borda properties (defined in the article)


In [ ]:
import os
import sys
sys.path.insert(0, os.path.join('.', 'fairdiv'))

import fairdiv
import fairdiv.algorithm as algorithm
import fairdiv.statistics as stats
import fairdiv.properties as props


# Create some constants
NUMBER_OF_GOODS = 8

# Create items
items = [fairdiv.Good(i+1) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[4], items[7], items[1], items[2], items[3], items[5], items[0], items[6]])
]

# Generate all allocations for the given problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))


results = {
    "OS": algorithm.original_sequential(agents, items),
    "RS": algorithm.restricted_sequential(agents, items),
    "SD": algorithm.singles_doubles(agents, items),
    "BU": algorithm.bottom_up(agents, items),
    "TR": algorithm.trump_algorithm(agents, items)
}

for k, v in results.items():
    print("{}: {}".format(k, v))

os_stats = stats.Statistics(A, agents, {
    "BPareto": props.is_borda_pareto,
    "BMaxSum": props.is_maximal_borda_sum,
    "BEnvyFree": lambda X, A, M: props.is_borda_envy_free(X, M),
    "BMaxMin": props.is_borda_max_min,
    "OLEnvyFree": lambda X, A, M: props.is_envy_free_ordinally(X, M),
    # "OLPareto": props.is_pareto_ordinally # Will take a long time
})
for X in results["OS"]:
    os_stats.add(X)

print(os_stats.formatted_text())

TR: [{B: (4, 5, 6, 8), A: (1, 2, 3, 7)}, {B: (2, 5, 6, 8), A: (1, 3, 4, 7)}]
OS: [{B: (3, 5, 6, 8), A: (1, 2, 4, 7)}]
BU: [{B: (3, 5, 6, 8), A: (1, 2, 4, 7)}, {B: (2, 5, 6, 8), A: (1, 3, 4, 7)}]
SD: [{B: (2, 5, 6, 8), A: (1, 3, 4, 7)}, {B: (2, 3, 5, 8), A: (1, 4, 6, 7)}, {B: (3, 4, 5, 8), A: (1, 2, 6, 7)}, {B: (3, 5, 6, 8), A: (1, 2, 4, 7)}, {B: (2, 4, 5, 8), A: (1, 3, 6, 7)}]
RS: [{B: (3, 5, 6, 8), A: (1, 2, 4, 7)}]


## Benchmarking algorithms
In this section, we are going to benchmark all algorithms to study produced allocations.

### 4 items problems


In [2]:
import os
import sys
sys.path.insert(0, os.path.join('.', 'fairdiv'))

import fairdiv
import fairdiv.algorithm as algorithm
import fairdiv.statistics as stats
import fairdiv.properties as props
import fairdiv.problemGenerators as generators

# Create some constants
NUMBER_OF_GOODS = 8

# Generate all possible problems (ie, all preferences) for 4 items
problems = generators.generate_possible_problems(4)

# Create items
items = [fairdiv.Good(i+1) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[4], items[7], items[1], items[2], items[3], items[5], items[0], items[6]])
]

# Benchmark algorithms
benchmark = stats.Benchmark(
    problems,
    {
        "OS": algorithm.original_sequential,
        "RS": algorithm.restricted_sequential,
        "SD": algorithm.singles_doubles,
        "BU": algorithm.bottom_up,
        "TR": algorithm.trump_algorithm
    },
    {
        "BPareto": props.is_borda_pareto,
        "BMaxSum": props.is_maximal_borda_sum,
        "BEnvyFree": lambda X, A, M: props.is_borda_envy_free(X, M),
        "BMaxMin": props.is_borda_max_min,
        "BNash": props.is_borda_nash,
        # "BEgalitarian": lambda X, A, M: props.is_borda_egalitarian(X, M),
        "OLEnvyFree": lambda X, A, M: props.is_envy_free_ordinally(X, M),
        # "OLPareto": props.is_pareto_ordinally # Will take a long time,
        "Max-min": props.is_max_min
    }
)

print(benchmark.run())


{'RS': {'[2, 1, 3, 0]': Allocations:
	Allocation: {B: (2, 3), A: (0, 1)}
		BMaxMin: True
		Max-min: True
		BMaxSum: True
		BPareto: True
		BNash: True
		OLEnvyFree: True
		BEnvyFree: True
, '[2, 0, 1, 3]': Allocations:
	Allocation: {B: (2, 3), A: (0, 1)}
		BMaxMin: True
		Max-min: True
		BMaxSum: True
		BPareto: True
		BNash: True
		OLEnvyFree: True
		BEnvyFree: True
	Allocation: {B: (1, 2), A: (0, 3)}
		BMaxMin: True
		Max-min: True
		BMaxSum: False
		BPareto: True
		BNash: False
		OLEnvyFree: True
		BEnvyFree: True
, '[0, 2, 1, 3]': Allocations:
	Allocation: {B: (2, 3), A: (0, 1)}
		BMaxMin: False
		Max-min: True
		BMaxSum: True
		BPareto: True
		BNash: True
		OLEnvyFree: False
		BEnvyFree: False
	Allocation: {B: (0, 3), A: (1, 2)}
		BMaxMin: True
		Max-min: True
		BMaxSum: False
		BPareto: True
		BNash: False
		OLEnvyFree: True
		BEnvyFree: True
	Allocation: {B: (0, 2), A: (1, 3)}
		BMaxMin: False
		Max-min: True
		BMaxSum: True
		BPareto: True
		BNash: True
		OLEnvyFree: False
		BE